In [ ]:
#Libraries
import csv
from numpy import *
import matplotlib.pyplot as plt
from scipy import optimize

#Identify Filepath
print("How many plates are sheilding the peak?")
number = input()
print("Which peak are you looking at? 1 or 2?")
peak = input()
if (peak == "1"): filepath = 'D:\\Sussex\\Year 3\\Advanced Physics Laboratory\\NaI Scintillator\\Useful Images\\Peaks\\Plate ' + number + '\\60Co 1173.2keV\\Peak';
else: filepath = 'D:\\Sussex\\Year 3\\Advanced Physics Laboratory\\NaI Scintillator\\Useful Images\\Peaks\\Plate ' + number + '\\60Co 1332.5keV\\Peak';


#Provide Estimates
print("Estimate the Standard Deviation of the peak")
sest = float(input())
print("Estimate the Mean of the peak")
mest = float(input())
print("Estimate the area of the peak")
fest = float(input())
#Open .csv File
print ("opening : " + filepath)
datafile = open(filepath + ".csv", 'r');
datareader = csv.reader(datafile, delimiter=',');
file = [];

#Calibration Settings
m = 1.01E+00
c = 2.79E+01

#Convert File into two arrays
for row in datareader:
    file.append(row);
channel = [];
count = [];
for row in file:
    channel.append(float(row[0]));
    count.append(m*float(row[1]) + c);

#Define Functions
def gaussian(x,s,m,f): #Flat Gaussian (not on a slope)
    return f/(s*sqrt(2*pi))*exp((x-m)**2/(-2*s**2));
def leaningGaussian(x,s,m,f,a,b): #Gaussian on a slope
    return f/(s*sqrt(2*pi))*exp((x-m)**2/(-2*s**2)) + x*a + b;
def linear(x,a,b): #Linear graph
    return a*x + b;

#Plot & Save Raw Data
image = plt.figure(num=None, figsize=(20, 6), dpi=1200, facecolor='w', edgecolor='k')
plt.plot (channel, count);
plt.grid(True)
plt.title("Raw Peak Data for 60-Co Peak #" + str(peak) + " shielded by " + str(number) + " plates")
plt.xlabel(r'Energy [keV]')
plt.ylabel(r'Counts []')
#image.savefig(filepath +" Raw"+ '.png')

#Estimate the Standard Deviation, Mean and Area (On a flat gaussian)
gaussCoeff, gaussCov = optimize.curve_fit(gaussian,channel,count,p0 = [sest,mest,fest]);
s,m,f=gaussCoeff
olds,oldm,oldf = gaussCoeff

#Refine Estimates with Gaussian on a Slope
gaussCoeff, gaussCov = optimize.curve_fit(leaningGaussian,channel,count,p0 = [s,m,f,0,0]);
s,m,f,a,b = gaussCoeff

#Create Smooth-Esque range
x = arange(min(channel),max(channel),1)

#Plot Sloped Gaussian
image = plt.figure(num=None, figsize=(20, 6), dpi=1200, facecolor='w', edgecolor='k')
plt.plot (channel, count, 'kx');
plt.plot(x,leaningGaussian(x,s,m,f,a,b));
plt.plot(x,linear(x,a,b));
plt.grid(True);
plt.title("Smoothed Peak on a slope for 60-Co Peak #" + str(peak) + " shielded by " + str(number) + " plates")
plt.xlabel(r'Energy [keV]')
plt.ylabel(r'Counts []')
image.savefig(filepath +" LR"+ '.png')

#Plot Background Corrected Gaussian
image = plt.figure(num=None, figsize=(20, 6), dpi=1200, facecolor='w', edgecolor='k')
plt.plot(x,gaussian(x,s,m,f))
plt.grid(True)
plt.title("Smooth Corrected Peak for 60-Co Peak #" + str(peak) + " shielded by " + str(number) + " plates")
plt.xlabel(r'Energy [keV]')
plt.ylabel(r'Counts []')
#image.savefig(filepath +" Corrected"+ '.png')

#Plot Residuals
image = plt.figure(num=None, figsize=(20, 6), dpi=1200, facecolor='w', edgecolor='k')
model = [leaningGaussian(i,s,m,f,a,b) for i in channel]
plt.errorbar(channel,[i-j for i,j in zip(count,model)],, 'kx')
plt.grid(True)
plt.title("Residuals for 60-Co Peak #" + str(peak) + " shielded by " + str(number) + " plates")
plt.xlabel(r'Energy [keV]')
plt.ylabel(r'Counts []')
image.savefig(filepath +" Residuals"+ '.png')


#Write Curve Details and Statistics to .txt File
file = open(filepath + " Details.txt","w")

file.write("the program guessed the origional peak had a \n")
file.write("Standard deviation of : " + str(olds) + "\n")
file.write("Peak Mean of : " + str(oldm) + "\n")
file.write("Scalar Multiplier of : " + str(oldf) + "\n")
file.write("\n")
file.write("the algorithm then refined this estimate to find the accurate values that" + "\n")
file.write("Standard deviation is :" + str(s) + "\n")
file.write("Peak Mean is at : " + str(m) + "\n")
file.write("Peak Area is : " + str(f) + "\n +- " + str(sqrt(gaussCov[2,2])))
file.write("Background Slope is : " + str(a) + "\n")
file.write("Background intercept is : " + str(b) + "\n \n \n")
file.write(str(gaussCov))

file.close()
print("Finished")

How many plates are sheilding the peak?
0
Which peak are you looking at? 1 or 2?
1
Estimate the Standard Deviation of the peak
35
Estimate the Mean of the peak
1200
Estimate the area of the peak
26000
